#### Notebook config

In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%autoreload 2

In [7]:
import fwipy 
print(fwipy.__file__)
from fwipy import *
from fwipy.config.logging import *
lll(40)
from fwipy.ioapi.segy import SgyFile
plt.style.reload_library()
plt.style.use(['default', 'paper2022kolumbo'])

/home/kajetan/software/Kolumbo-FWI/fwipy/__init__.py


# Initalise data

In [9]:
data_path = '../data/'
# sgyobs = SgyFile('data_kolumbo_PhD_all_traces_field_fltrd.sgy', data_path)
# sgy000 = SgyFile('data_kolumbo_PhD_all_traces_synth_it000.sgy', data_path)
# sgylvz = SgyFile('method_s01_800_50-Synthetic.sgy', data_path)
# sgy020 = SgyFile('data_kolumbo_PhD_all_traces_synth_it020.sgy', data_path)
# sgy080 = SgyFile('data_kolumbo_PhD_all_traces_synth_it080.sgy', data_path)
# sgy160 = SgyFile('data_kolumbo_PhD_all_traces_synth_it160.sgy', data_path)
sgyobsm = SgyFile('data_kolumbo_PhD_all_traces_field_fltrd_muted.sgy', data_path)
sgy000m = SgyFile('data_kolumbo_PhD_all_traces_synth_it000_muted.sgy', data_path)
# sgy020m = SgyFile('data_kolumbo_PhD_all_traces_synth_it020_muted.sgy', data_path)
# sgy080m = SgyFile('data_kolumbo_PhD_all_traces_synth_it080_muted.sgy', data_path)
sgy160m = SgyFile('data_kolumbo_PhD_all_traces_synth_it160_muted.sgy', data_path)
sgylvzm = SgyFile('method_s01_800_50-Synthetic_muted.sgy', data_path)

In [12]:
# read bathymetry and final model 
from arrau.a2d import Arr2d
xp = PROTEUS()
xp.read_all()
b01 = Box3d(8,25, -3,15, 0,4)
bt = Arr2d(xp.read_bathy_topo().extract(np.array(b01.xy)*1e3).arr/1e3, extent=b01.xy)
from fwipy.ndat.arrays import Arr3d
vp = Arr3d(source='../data/p14-CP00160-Vp.mmp', shape=(341,361,81), extent=b01.extent)
from arrau.a3d import Arr3d
vp = Arr3d(vp / 1000, extent=b01.extent)
sl = vp.slice(value=2.55, axis=2, unit='km')

# Define plotters

In [10]:
from fwipy.utils import *
sgydict = {'obsm': {'name': 'field', 'file': sgyobsm},
           'lvzm': {'name': 'synth_start_LVZ', 'file': sgylvzm},
           '000m': {'name': 'synth_start', 'file': sgy000m},
           '160m': {'name': 'synth_final', 'file': sgy160m},
          }
pairs = [['lvzm', '000m'], ['obsm', '000m'], ['obsm', '160m']]
def plot_model_etc(sgyfile, tracf, ep, cmap='viridis_r'):
    sl.plot(mode='im', cmap=cmap, vmin=3.4, vmax=5,
            colors='Grey', linestyles='solid', levels=np.arange(3.5,5.6,0.5),
            show_cntr_labels=True, fontsize=8, fmt='%1.1f',
            orientation='horizontal', position='bottom', pad=.7, label='P-wave velocity (km/s)')
    bt.plot(mode='contour', aspect='equal', colors='k')  

    srcs_x, srcs_y, recs_x, recs_y = read_geom(sgyfile, tracf, divide_by=1000)
    x1, x2, y1, y2 = read_geom_bounds(sgyfile, divide_by=1000, pady=600)
    shotcolor='k'
    bt.plot(mode='contour', aspect='equal', colors='w')   
    plt.plot(srcs_x, srcs_y, linestyle='', marker='.', label='shot',
                 markersize=1, markerfacecolor=shotcolor, markeredgecolor=shotcolor)
    plt.plot(recs_x, recs_y, marker='o', linestyle='', markersize=5, 
                       markeredgecolor='k', markeredgewidth=1, markerfacecolor='b')
    for x, y in zip(recs_x, recs_y):
        plt.gca().annotate(text=str(tracf), xy=(x,y), xytext=(x, y+0.5), clip_on=True, fontsize=8,
                            bbox={'facecolor': 'w', 'edgecolor': 'k', 'alpha': .5})   
    circ = plt.Circle((recs_x[0],recs_y[0]), 5, color='Grey', alpha=.5)
    plt.gca().add_patch(circ)
    plot_box_around_line(sgyfile, tracf, ep, divide_by=1000, padx=0, pady=200)
    plt.xlim(x1, x2)
    plt.ylim(y1, y2)
    plt.xlabel('X (km)')
    plt.ylabel('Y (km)')
def plot_all(sgydict, sgyid1, sgyid2, tracf, ep, save, cmap='cmo.rain_r',
             ins_xlim=[14e3, 18e3], ins_ylim=[1,3], 
             ins_pos=[0.25,1], ins_size=[.5, .5], # both in % of parent axes
            ):
    from matplotlib.gridspec import GridSpec
    file1 = sgydict[sgyid1]['file']
    file2 = sgydict[sgyid2]['file']
    label1 = sgydict[sgyid1]['name']
    label2 = sgydict[sgyid2]['name']
    fig = plt.figure(figsize=(10,16))
    gs = GridSpec(2, 1, height_ratios=[1,4])
    # data
    ax1 = fig.add_subplot(gs[0,0])
    kws = dict(ow=0, norm='rms', amplify=10, ax=ax1, vel_red=5000,
               tmin=1, tmax=3)
    ax1 = plot_data_wigg(file1, tracf, ep, label=label1, color='r', **kws)
    ax1 = plot_data_wigg(file2, tracf, ep, label=label2, color='k', **kws)
    ax1.invert_yaxis()
    plt.legend(loc='lower left')
    # inset 
    axins = ax1.inset_axes(ins_pos + ins_size)
    plot_data_wigg(file1, tracf, ep, label=label1, color='r', **dict(kws, ax=axins))
    plot_data_wigg(file2, tracf, ep, label=label2, color='k', **dict(kws, ax=axins))
    # sub region of the original image
    axins.set_xlim(ins_xlim)
    axins.set_ylim(ins_ylim)
    axins.set_xlabel('')
    axins.set_ylabel('')    
    axins.set_xticks([])
    axins.set_yticks([])
    axins.set_xticklabels([])
    axins.set_yticklabels([])
    axins.invert_yaxis()
    _ = ax1.indicate_inset_zoom(axins, edgecolor="k", lw=2)    
    
    # model etc. 
    ax2 = fig.add_subplot(gs[1,0])
    plot_model_etc(sgydict[sgyid1]['file'], tracf=tracf, ep=ep, cmap=cmap)
    plt.tight_layout()
    if save:
        add = 'ins_'
        plt.savefig('figures/suppl/fit_%sOBS%s_line%s_%s_vs_%s.png' % (add, tracf, 
                                                                  str(ep).rjust(3, '0'), 
                                                                  label1, 
                                                                  label2),
                    dpi=300)
        plt.close()
plt.rcParams.update({'font.size': 14})
# plot_all(sgydict, 'obsm', '160m', tracf=4105, ep=30, save=0, ins_size=[.5, .5],
#          ins_xlim=[17e3, 20e3], ins_ylim=[1.25, 2.35], ins_pos=[.25,-0.75])

# OBS 112, line 30

In [74]:
%%time 
for pair in pairs:
    plot_all(sgydict, *pair, tracf=4112, ep=30, save=1, ins_size=[.5, .5],
         ins_xlim=[16e3, 20e3], ins_ylim=[1.25, 2.35], ins_pos=[.25,-0.75])

CPU times: user 19.8 s, sys: 567 ms, total: 20.4 s
Wall time: 24.5 s


# OBS 110, line 7

In [9]:
bounds = read_geom_bounds(sgy000m, 1000)

In [10]:
bounds

(8.051, 24.996, -2.126, 13.847)

In [13]:
%%time 
for pair in pairs:
    plot_all(sgydict, *pair, tracf=4110, ep=7, save=1, ins_size=[.5, .5],
         ins_xlim=[8051, 13e3], ins_ylim=[1.25, 2.35], ins_pos=[.25,-0.75])

CPU times: user 22.5 s, sys: 921 ms, total: 23.4 s
Wall time: 36.1 s


# OBS 137, line 31

In [15]:
%%time 
for pair in pairs:
    plot_all(sgydict, *pair, tracf=4137, ep=31, save=1, ins_size=[.5, .5],
         ins_xlim=[8051, 13e3], ins_ylim=[1.1, 2.2], ins_pos=[.25,-0.75])

CPU times: user 20.1 s, sys: 628 ms, total: 20.7 s
Wall time: 29 s


# All well-fitted

In [8]:
# %%time 
te = {112: [30,27], 110: [7], 105: [27,26,9], 138: [5,6,31], 137:[31], 
      139:[38,37,31,6], 125: [30,5], 124:[37], 123:[27,6]}

In [14]:
good = {
    139: {
            38:  {'xlim': [12e3, 17e3], 'ylim': [1.5, 2.5]},
            37:  {'xlim': [14e3, 24e3], 'ylim': None},
            31:  {'xlim': [14e3, 23e3], 'ylim': None},
            6:   {'xlim': [13e3, 24e3], 'ylim': None},
        },
    138: {
            31:  {'xlim': [10e3, 14e3], 'ylim': None},
            6:   {'xlim': [10e3, 14e3], 'ylim': None},
            5:   {'xlim': [10e3, 14e3], 'ylim': None},
        },    
    137: {
            31:  {'xlim': [9e3, 14e3], 'ylim': [1.1, 2.2]},
        },        
    125: {
            30:  {'xlim': [15e3, 19e3], 'ylim': [1.5, 2.5]},
            5:   {'xlim': [16e3, 21e3], 'ylim': None},
        },    
    124: {
            37:   {'xlim': [8051, 13e3], 'ylim': None},
        },   
    123: {
            37:   {'xlim': [8051, 13e3], 'ylim': None},
            6:   {'xlim': [8051, 13e3], 'ylim': None},
        },  
    112: {
            30:   {'xlim': None, 'ylim': None},
            27:   {'xlim': None, 'ylim': None},
        },  
    110: {
            7:   {'xlim': None, 'ylim': None},
        },  
    105: {
            27:  {'xlim': [12e3, 16e3], 'ylim': [1.5, 2.5]},
            26:  {'xlim': [12e3, 16e3], 'ylim': None},
            9:   {'xlim': [11e3, 17e3], 'ylim': None},
        },      
}

In [22]:
%%time 
for tracf, lines in good.items():
    for ep, lims in lines.items():
        xlim = [16e3, 20e3] if lims['xlim'] is None else lims['xlim']
        ylim = [1.25, 2.5] if lims['ylim'] is None else lims['ylim']
        for pair in pairs:
            plot_all(sgydict, *pair, tracf=4000+tracf, ep=ep, save=1, ins_size=[.5, .5],
                 ins_xlim=xlim, ins_ylim=ylim, ins_pos=[.25,-0.75])

CPU times: user 6min 24s, sys: 26.4 s, total: 6min 50s
Wall time: 10min


# ...